# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [4]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木板路穿越一片广阔的草地，通向远处的开阔天空。图中，草地呈现出鲜艳的绿色，周围散布着些许灌木和树木。天空中有云彩，呈现出宁静淡定的夏日氛围。木板路引导观察者的视线深入画面，加强了这种向往自然和探索未知的感觉。整个场景充满了平静和宁静的氛围，令人感受到大自然的美丽与和谐。这样的场景可能位于公园或自然保护区，是人们散步和放松的理想场所。', role='assistant', function_call=None, tool_calls=None))


In [5]:
response.choices[0].message.content

'这幅图展示了一条木板路穿越一片广阔的草地，通向远处的开阔天空。图中，草地呈现出鲜艳的绿色，周围散布着些许灌木和树木。天空中有云彩，呈现出宁静淡定的夏日氛围。木板路引导观察者的视线深入画面，加强了这种向往自然和探索未知的感觉。整个场景充满了平静和宁静的氛围，令人感受到大自然的美丽与和谐。这样的场景可能位于公园或自然保护区，是人们散步和放松的理想场所。'

### 封装成一个函数 query_image_description

In [6]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [7]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一副幽默的对比图，展示了两只不同风格描绘的狗。图中左边的狗被描绘成一种非常夸张的肌肉型样式，有着人体的肌肉结构和姿态，头部是一只金毛犬的头。右边的狗则看起来更加常见一些，是一只坐着的柴犬，呈现一种平和而稍显悲伤的表情。

左边的图片标题为“16岁的我”，配文有“我前途一片光明”、“未来可期”等，表达了青少年时期充满希望和梦想的心境。右边的图片标题为“工作后的我”，配文则是“好累好累”、“腰疼腿疼”等，反映了进入职场后常见的疲惫感和身体不适。

总的来说，这幅图通过幽默的方式对比


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [10]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [11]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图显示了1980年到2020年间美国、中国、日本和德国国内生产总值(GDP)的比较。横坐标表示年份，纵坐标表示GDP（以万亿美元计算）。每个国家用不同的颜色表示其GDP走势。

- 蓝色线表示美国的GDP，可以看到其在这段时间内持续增长，特别是从1995年开始加速上升。
- 红色线表示中国的GDP，表现出了非常显著的增长趋势，特别是从2000年以后，其增长速度迅猛，显著地接近美国。
- 紫色线表示日本的GDP，增长较为平稳，在1995年到2005年间略有波动，之后趋于稳定。
- 绿色线表示德国的GDP，整体增长较为缓慢和平稳，与日本相似。

总体上，这幅图展示了四个经济体在过去四十年的经济表现，尤其突出了中国的快速增长。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [ ]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [ ]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

![](./images/handwriting_1.jpg)

In [16]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示了一个笔记本上的笔记内容，主要涉及到深度学习中的模型调整技术与相关方法。从左侧页面可以看出，笔记围绕着transformer模型，特别是关于其参数调整方法如Prompt Tuning，Adapters等。右侧页面则更专注于多模态、指令型微调（如LaMDA, LoRA, PET等）。

具体来说，左侧提到了几种不同的调整技术：
- **Adapter**: 例如Google在2019年提出的Adapter。
- **Prefix**: 来自斯坦福大学2021年的方法。
- **Prompt**: Google在2021年的方法。
- 还列举了其他技术如"Soft prompts"和"Hand prompts"。

右侧页面则从多模态指令型微调的角度出发，探讨了不同的技术，并提及了一些具体的框架和模型，如Lora和PETC。

这些笔记反映出作者在探索NLP（自然语言处理）领域的最新技术和方法，特别是在transformer模型调整方面，展示出针对特定任务进行模型优化的多种策略。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

# 1.1 GPT-4V.jpg

![img_temp](./images/gpt-4v.jpg)

In [15]:
from IPython.display import display, Markdown

content = query_base64_image_description("./images/gpt-4v.jpg", prompt="请对图片进行分析描述，并回答图片上的问题")
display(Markdown(content))

图片展示的是一只黄色的卡通动物，看起来像是一只兔子，具有蓝色大眼睛和粉色的脸颊。它的表情似乎呈现出一种微笑和好奇的混合状态。图片上的字很难完全辨认，但看起来像是手写的中文，大致可以猜测写的是“这是什么动物？” 。

答案是，这看起来是一个设计有趣的卡通动物，但它并不符合现实中的任何特定动物。这种类型的卡通形象通常是为了娱乐和艺术创造而设计的，不一定对应现实世界中的任何具体物种。